# Binary Fingerprint Generation

This notebook has the purpose of generating a binary fingerprint for each probe-burst, starting from the binary string extracted from the probe, and by applying the best filters and related thresholds, in order to compress said strings.

## Libraries and Configurations

Import configuration files

In [152]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [153]:
import pandas as pd

Import **other libraries**

In [154]:
from rich.progress import Progress
from rich import traceback

traceback.install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7954ee982950>>

## Functions

In [155]:
def bitwise_and(bit_str1: str, bit_str2: str) -> str:
    # Convert bit strings to integers
    int1 = int(bit_str1, 2)
    int2 = int(bit_str2, 2)

    # Perform bitwise AND operation
    result = int1 & int2

    # Convert result back to binary string
    result_str = bin(result)[2:]  # [2:] to remove '0b' prefix

    # Return result
    return result_str.zfill(max(len(bit_str1), len(bit_str2)))

In [156]:
def sumFilter(bitwise_and: str) -> int:  # decimal - converted sum filter
    sum = 0
    a = 0
    for i in bitwise_and:
        sum += (2**a) * int(i, 2)
        a += 1
    return sum

In [157]:
burst = "000000001111111100000000"
fltr = "000000001110000000000000"
thrs = 4

In [158]:
if len(burst) != len(fltr):
    raise ValueError("Burst and filter lengths do not match")

In [159]:
filtered = sumFilter(bitwise_and(burst, fltr))

In [160]:
filtered

1792

In [161]:
fingerprint = {}

In [162]:
if filtered < thrs:
    fingerprint[(burst)] = -1
elif filtered >= thrs:
    fingerprint[(burst)] = 1

In [163]:
fingerprint

{'000000001111111100000000': 1}

In [164]:
string_df_csv = pd.read_csv("../../data/interim/string_df.csv", index_col=0)

In [165]:
strings_df = pd.DataFrame(string_df_csv)

In [166]:
strings_df.rename(columns={list(strings_df)[0]: "Bursts"}, inplace=True)

In [167]:
strings_df

,Bursts
0,0000000000000100000000100000010000001011000101...
1,0000000000000100000000100000010000001011000101...
2,0000000000000100000000100000010000001011000101...
3,0000000000000100000000100000010000001011000101...
4,0000000000000100000000100000010000001011000101...
...,...
325,0000000000000100000000100000010000001011000101...
326,0000000000000100000000100000010000001011000101...
327,0000000000000100000000100000010000001011000101...
328,0000000000000100000000100000010000001011000101...


In [168]:
best_configs_csv = pd.read_csv("../experiments_results/best_configs.csv")

In [169]:
bests_configs_df = pd.DataFrame(best_configs_csv)

In [170]:
bests_configs_df

,Filter,Threshold
0,0000000000000000000000000000000000000000000000...,32
1,0000000000000000000000000000000000000000000000...,32


In [171]:
fingerprint_df = pd.DataFrame(columns=["Bursts", "Fingerprint"])

In [172]:
fingerprint_df

,Bursts,Fingerprint
